# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key
geoapify_key

'fc6428dce12f459a9ac1d636f5f41121'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")
#city_data_df
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,299.32,80,89,4.20,MU,1701785385
1,1,ancud,-41.8697,-73.8203,287.48,73,1,2.21,CL,1701785343
2,2,puerto armuelles,8.2833,-82.8667,302.13,68,0,1.34,PA,1701785221
3,3,margaret river,-33.9500,115.0667,288.25,79,50,6.94,AU,1701785007
4,4,keflavik,64.0049,-22.5624,275.39,75,40,7.72,IS,1701785387


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"    
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_2 = city_data_df.loc[city_data_df["Cloudiness"] < 75]
#city_data_df_2
# Drop any rows with null values
city_data_df_3 = city_data_df_2.dropna()
city_data_df_3
# Display sample data
map_plot_2 = city_data_df_3.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Cloudiness",
    color = "City"    
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Cloudiness)

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
pd.set_option('mode.chained_assignment', None)
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,80,
1,ancud,CL,-41.8697,-73.8203,73,
2,puerto armuelles,PA,8.2833,-82.8667,68,
3,margaret river,AU,-33.9500,115.0667,79,
4,keflavik,IS,64.0049,-22.5624,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set the geographical coordinates
latitude = 40.791439777608495
longitude = -74.1987781490682

# Set parameters to search for a hotel
limit = 20
radius = 10000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"][index]
    longitude = hotel_df["Lng"][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
ancud - nearest hotel: Cabañas
puerto armuelles - nearest hotel: Coco del Mar
margaret river - nearest hotel: Margaret River Hotel
keflavik - nearest hotel: Núpan Deluxe
tolanaro - nearest hotel: Hôtel Mahavokey
tiksi - nearest hotel: Арктика
west island - nearest hotel: Cocos Village Bungalows
pariaman - nearest hotel: HOTEL NAN TONGGA
tura - nearest hotel: No hotel found
yellowknife - nearest hotel: 5505 45 Str eet
albany - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Chemin de Keravel
hawaiian paradise park - nearest hotel: No hotel found
rolla - nearest hotel: Pear Tree Inn Rolla
kavaratti - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
ribeira grande - nearest hotel: Hospedaria JSF
isafjordur - nearest hotel: Hótel Horn
adamstown - nearest hotel: No hotel found
tlahualilo de zaragoza - nearest hotel: No hote

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port mathurin,MU,-19.6833,63.4167,80,Escale Vacances
1,ancud,CL,-41.8697,-73.8203,73,Cabañas
2,puerto armuelles,PA,8.2833,-82.8667,68,Coco del Mar
3,margaret river,AU,-33.9500,115.0667,79,Margaret River Hotel
4,keflavik,IS,64.0049,-22.5624,75,Núpan Deluxe
...,...,...,...,...,...,...
558,lincoln city,US,44.9582,-124.0179,96,Inn at Lincoln City
559,pyapon,MM,16.2833,95.6833,78,No hotel found
560,san jose de las matas,DO,19.3391,-70.9382,51,Hotel La Mansion
561,severobaykal'sk,RU,55.6500,109.3167,79,Черембасс


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_3 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover_cols = ["Country","Hotel Name"]    
)

# Display the map
map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)